# Networks Exercise

In [ ]:
# Imports 
import sys
from py2neo import authenticate, Graph, Relationship
#from py2neo.cypher import CypherWriter
#import MySQLdb
import pandas
import pymysql
import numpy as np
import operator 
import pprint

print "--COMPLETE--"

## Connect to the Neo4j Database
Run the cell below to set up a connection to the Neo4j database using py2neo

In [ ]:
# set up authentication parameters
# replace password
authenticate("localhost:7474", "neo4j", "<PASSWORD>")

# Create a variable for our graph and print our connection infomation
graph_db = Graph()
print graph_db


#### The cell below is intended for you to test Cypher Queries

# Introduction to Neo4j and Cypher
## What is a Graph Database?

### Quick Review of Neo4j and Cypher 

When you think of data, you probably imagine an excel table where each row is an individual observation or data point. 
For example, 

|name | age | employee_id|
|-----|-----|------------|
|Joe  |  34 |   12345    |
|Ann  |  54 |   12346    |
    
In a graph database, that row is more like a ball. 


##### (node: Employee {name:Joe, age: 34, employee_id: 12345})

This ball is called a "node" in Neo4j. In fact, the little parentheses around Joe's information are designed to help the user think of Joe's information as a little ball. 

All the information about Joe is still there, but it's just not in a flat table format. Instead of storing each peice of Joe's information in a variable column, Joe's information is stored as **properties**, ie name, age, employee_id.
Joe's node also has a **label**, *Employee*, to identify this node as belonging to an employee.


Neo4j uses the Cypher Query Language to get information out of the Database. 

##### Components of a simple Cypher Query:
+ MATCH      
     - Essentially the same thing as SELECT in SQL 
+ (n)        
    - Any node (the n is just a variable, could be any letter) 
+ RETURN     
    - Needed in every query 
+ LIMIT      
    - Same as in SQL 

The following query will return 20 nodes from the database:  
##### graph_db.cypher.execute("MATCH (n) RETURN n LIMIT 20")

If I wanted to query just Award nodes, I would run this query;  
##### graph_db.cypher.execute("MATCH (n:Award) RETURN n LIMIT 20")


## Part A - Cypher Query language

### Write a query that matches 10 nodes of type Employee and returns the employeeid. Limit to 10 nodes.

In [ ]:
graph_db.data("MATCH (n:Employee) RETURN n.employeeid LIMIT 10")

In [ ]:
"""
Your query should return a list containing dictonaries.

Returned Object from py2neo

[{u'n.employeeid': u'90014540'},
 {u'n.employeeid': u'90026679'},
 {u'n.employeeid': u'90017693'},
 {u'n.employeeid': u'90028696'},
 {u'n.employeeid': u'90013819'},
 {u'n.employeeid': u'90001118'},
 {u'n.employeeid': u'90001300'},
 {u'n.employeeid': u'90023086'},
 {u'n.employeeid': u'90017074'},
 {u'n.employeeid': u'90025003'}]
"""
print "________"

What makes graph databases like Neo4j special is the ability to create relationships between nodes:

For instance, the database has two types of relationships; WORKED_ON  and WORKED_WITH
 + Employees have a *WORKED_ON* relationship with nodes which represent awards on which the person has worked.   
 + Employees have a *WORKED_WITH* relationship other persons that have worked on the same award.   

To view what this looks like, this query will return 5 instances of the pattern where an employee worked_on an award, and return the relationship. 

#### graph_db.cypher.execute("MATCH (e)-[r:WORKED_ON]->(a) RETURN  r LIMIT 5;")

If you look at this query you kinda see a pattern of (node1) -[relationship]-> (node2). You can imagine that this is one ball with an arrow pointing to another ball. Each ball is a node of data and the arrow is a relationship.  What this query is asking the the database to do is go through all the nodes and relationships and find the pattern where one node has a realtionship pointing to another node and that realtionship has the label "WORKED_ON".  


### Write a query that matches nodes of type Employee that have a WORKED_WITH relationship with another node of type Employee. return the employeeid of both employee nodes. Limit the response to 5.  

In [ ]:
graph_db.data("MATCH (a)-[r:WORKED_WITH]->(b) RETURN a.employeeid, b.employeeid LIMIT 5;") 

In [ ]:
"""
Your query should return a list containing dictonaries.

[{u'a.employeeid': u'90000919', u'b.employeeid': u'90028696'},
 {u'a.employeeid': u'90024693', u'b.employeeid': u'90013819'},
 {u'a.employeeid': u'90001652', u'b.employeeid': u'90001118'},
 {u'a.employeeid': u'90025754', u'b.employeeid': u'90001300'},
 {u'a.employeeid': u'90004472', u'b.employeeid': u'90023086'}]

"""
print "_______"



## Neo4j and Networkx 

This would all be so much cooler if you could see the data, right?

But first, we have to convert our neo4j data into a networkx object after the Cypher query is executed.  For example:

In [ ]:
#Imports that should really be at the top of the notebook 
import networkx
# Allows plots to be showed inline 
#import matplotlib as plt
%matplotlib inline


# Get ALL the Nodes and relationships 
# The below query translates to,
# find all nodes that have a relationship with another node (this is any node in our graph)
# Return all the data for the nodes and relationships that are found 
all_data = graph_db.data("MATCH (n1) -[r]-> (n2) RETURN n1, r, n2;") 

# print all_data

# The following code is used to keep track of employees vs. students vs. award numbers,
emps = {}
awards = {} 
students = {}
color_vals = []

for tuple in all_data:
    # print "tuple: ", tuple
    n1_keys = tuple["n1"].keys()
    if "position" in n1_keys:
        if tuple["n1"]["position"] == "FACULTY":
            emps[tuple["n1"]["employeeid"]] = 1
        else:
            students[tuple["n1"]["employeeid"]] = 1
    else:
        awards[tuple["n1"]["award_num"]] = 1
        

#Convert all_data to a dataframe object
all_data_df = pandas.DataFrame(all_data)

# Adding a 'weight' to see connections better
all_data_df['weight'] = .01

# Create a networkx graph of all the data 
all_data_g = networkx.from_pandas_dataframe(all_data_df, 'n1', 'n2', ['weight'])

# Color the nodes
for node in all_data_g.nodes(data =True):
    id = 0
    if "employeeid" in node[0].keys():
        id = node[0]["employeeid"]
    else:
        id = node[0]["award_num"]

    if id in emps:
        color_vals.append ('g')  # green
    elif id in students:
        color_vals.append ('y')  # yellow
    else:
        color_vals.append ('r')  # red
    
print "--COMPLETE--"

### Graph all the Data! 
Run the cell below to create a networkx graph of all the data

In [ ]:
# visualize the network       
networkx.draw(all_data_g, node_color=color_vals)

**Admittently, this is not the most informative graph.  There are too many nodes and connections to be able to see any patterners or insigths in the dataset. At best, you might be able to get a rough idea of the relative number of Employees, to awards to students **

### We just created a plot of all the Employees, Students, and Awards in the Graph.  Now let's practice plotting subgraphs. 
 

### Write a function that accepts a specific award number, finds all the Employees/Students who worked on that award, and return the data as a networkx graph. Also, have your function plot the graph. 



**Hint: The folowing line returns the employees and student who worked on the award aw1006**
MATCH (n) -[r:WORKED_ON] -> (a:Award {award_num:'aw1006'}) RETURN n,r

**Hint: you can pass parameters to a ipython magic call by placing the parameter in curly brakets, the cell below shows an example**


In [ ]:
# This is the employee id of an employee I would like to search for. It is saved into a parameter, employee_id_parameter 
employee_id_parameter = '90022882'

# This line searches for an employee where the employee id is provided by the parameter, employee_id_parameter 
# This query translates to,
# Find the Node of type Employee where the employee id is equal to the value in the
# variable employee_id_parameter. Return that nodes data. 
graph_db.data("MATCH (e:Employee {employeeid:" + "'" + employee_id_parameter + "'" +"}) RETURN e;") 

In [ ]:
def employee_peers(award_num):
    """
    Creates a networkx plot of employees and students that have worked on an award.
    
    Parameters
    ----------
    award_num - The award_num of a specifc award, ie aw1006
    
    Returns 
    --------
    A Networkx graph object of an award and the persons who worked on that award 
    
    """
    # Collect Employees that worked with that Employee 
 
    award = award_num
    # This Cypher Query translates to,
    # Fins the nodes that have a WORKED_ON relationship with an node of type Award that has the award_num of the value of the 
    # award variable. Return the employeeid and award number of the found nodes 
    query = graph_db.data("MATCH (n) -[r:WORKED_ON]-> (a:Award {award_num:" + "'" + award + "'" + "}) RETURN n.employeeid, a.award_num;") 
    
    # Convert to DataFrame
    node_data = pandas.DataFrame(query)
    cols = node_data.columns
    cols = cols.map(lambda x: x.replace('.', '_') if isinstance(x, (str, unicode)) else x)
    node_data.columns = cols
    
    pprint.pprint(node_data)

    # Create the Networkx Graph 
    G = networkx.from_pandas_dataframe(node_data, 'n_employeeid', 'a_award_num')
    

    # The following code is used to keep track of employees vs. award numbers,
    # and then to assign colors to nodes of each type in the graph
    emps = dict.fromkeys(node_data.n_employeeid.unique(), 1)
    awards = dict.fromkeys(node_data.a_award_num.unique(), 1)
    color_vals = []
    
    
    for node in G.nodes(data =True):
        if node[0] in emps:
            color_vals.append ('g')  # green
        else:
            color_vals.append ('r')  # red
        
    networkx.draw(G, node_color = color_vals)
    
    #Return the Graph Object 
    return G 
        
# Testing the function with the award aw1501
employee_peers("aw1501")

## Anaylsing Networks 
The networkx module also has built in tools to analyse graphs. The following measurements are used to assess how the nodes are related to each other in the graph database.

### Network Measurements 
Here is some common vocabularly when studing networks: 

 **Degree Centrality ** - counts the number of edges that a node has 
     - Nodes with a high degree of connections usually play an important role in a network 
 **Betweenness ** - indicator of a nodes centality in a network. 
     - Equal to the number of shortest paths from all vertices to all others that pass through that node 
 **Diameter** - The longest shrortest path over all pairs of nodes 
     - Often we want to find the shortest distance between two nodes; the diameter is the longest of theses paths 
     - Nodes that occur on many shortest paths between other nodes in the graph have a high betweenness centrality score 
 **Cliques ** - A clique is a subset of vertices of an undirected graph such that every two distinct vertices in the clique are adjacent.

The functions below are written for your conviencence by http://www.slideshare.net/arnicas/a-quick-and-dirty-intro-to-networkx-and-d3

     
   
 

In [ ]:
def calculate_degree_centrality(graph):
    ''' 
    Calculate degree centrality of a node, sets value on node as attribute; returns graph, and dict of the degree centrality values.
	Also has code to print the top 10 nodes by degree centrality to console
    
    Parameters
    ----------
    graph - A Networkx object Graph 
	'''
    g = graph
    dc = networkx.degree_centrality(g)
    networkx.set_node_attributes(g, 'degree_centrality', dc)
    degcent_sorted = sorted(dc.items(), key = operator.itemgetter(1), reverse = True)
    for key, vlaue in degcent_sorted[0:10]:
        print "Highest degree Centrality:", key, vlaue
    return graph, dc 

print "--COMPLETE--"

In [ ]:
def calculate_betweenness(graph):
    ''' 
    Calculate betweenness centrality of a node, sets value on node as attribute; returns graph, and dict of the betweenness centrality values

    Parameters
    ----------
    graph - A Networkx object Graph 
    '''
    g = graph
    bc=networkx.betweenness_centrality(g)
    networkx.set_node_attributes(g,'betweenness',bc)
    degcent_sorted = sorted(bc.items(), key = operator.itemgetter(1), reverse = True)
    for key, vlaue in degcent_sorted[0:10]:
        print "Highest Betweeness :", key, vlaue
    return g #, bc

print "--COMPLETE--"

In [ ]:
def find_cliques(graph):
	''' 
    Calculate cliques and return as sorted list.  Print sizes of cliques found.
    
    Parameters
    ----------
    graph - A Networkx object Graph 
	'''
	g = graph
	cl = networkx.find_cliques(g)
	cl = sorted(list( cl ), key=len, reverse=True)
	print "Number of cliques:", len(cl)
	cl_sizes = [len(c) for c in cl]
	print "Size of cliques:", cl_sizes
	return cl

print "--COMPLETE--"

In [ ]:
def find_diameter(graph):
    """
    Calculate the Diameter of a graph
    
    Parameters
    ----------
    graph - A Networkx object Graph    
    """
    diameter = networkx.diameter(graph)
    print "Diameter: ", diameter
    return diameter

print "--COMPLETE--"

## Part B(a) - Analysing Graphs 

### Preform some exploritory analysis on the full data. Use the funtions above to,
    1. Obtain a list of the highest 10 levels of Degree Centrality
    2. Obtain a list of the highest 10 levels of Betweenness
    3. The number of cliques and the sizes of those cliques. 
        
    
By looking at these results, what do you think about the shape of the graph? How would you charaterize the relationships in the graph? 

In [ ]:
print "_________________DEGREE CENTRALITY ________________"
calculate_degree_centrality(all_data_g)
print "_________________Betweenness ________________"
print calculate_betweenness(all_data_g)
print "_________________Cliques ________________"
find_cliques(all_data_g)
print "__________________________________________"

## Part B(b) - Analysing a Sub Graph 

From looking at the measures above, it seems like our graph has a lot of independent units that are not connected to the larger graph. For instances, there are a lot of cliques with length 2. This indicates one employee working on one award. Let's say we are intrested in the sub-network where all the nodes are connected. 

### Preform the same exploritory analysis on the subset created below. The subset is the largest connected set of nodes in the graph. 

Use the funtions above to,
    1. Obtain a list of the highest 10 levels of Degree Centrality
    2. Obtain a list of the highest 10 levels of Betweenness
    3. The number of cliques and the sizes of those cliques
    4. Find the Diameter of the Subgraph
    
By looking at these results, what do you think about the shape of the graph? how would you charaterized the relationships in the graph? 

In [ ]:
# Create the Subgraph 
subgraph_generator = max(networkx.connected_components(all_data_g), key =len)

subgraph_g = all_data_g.subgraph(subgraph_generator)

# Keep track of descriptive counters
num_awards = 0
num_faculty = 0
num_students = 0

# Color the nodes
color_vals = []
for node in subgraph_g.nodes(data =True):
    id = 0
    if "employeeid" in node[0].keys():
        id = node[0]["employeeid"]
    else:
        id = node[0]["award_num"]

    if id in emps:
        color_vals.append ('g')  # green
        num_faculty += 1
    elif id in students:
        color_vals.append ('y')  # yellow
        num_students += 1
    else:
        color_vals.append ('r')  # red
        num_awards += 1

networkx.draw(subgraph_g, node_color=color_vals)

person_data = list(networkx.get_node_attributes(subgraph_g,'position').values())

print 'Number of Awards: ', num_awards
print 'Number of Students: ', num_students
print 'Number of Faculty: ', num_faculty

In [ ]:
print "_________________DEGREE CENTRALITY ________________"
calculate_degree_centrality(subgraph_g)
print "_________________Betweenness ________________"
print calculate_betweenness(subgraph_g)
print "_________________Cliques ________________"
find_cliques(subgraph_g)
print "_________________Diameter ________________"
find_diameter(subgraph_g)
print "______________________________________________"

# Plotting the Results 

Betweeness and Centrality are both measures on how connnected a node is to its surounding nodes. However, they measure connectiveness in slightly different ways. A Node with high Degree centrality does not nessasarily mean the node also has a high level a Betwennness Centrality.

   A node with high betweenness but low degree might indicated that a node's few connections are critical to the shape of the graph

   A node with high degree but low betweenness might indicate that a node's connections are redundent and connections can bypass that node
    
    
### Run the Cell below to create a scatterplot of betweeness vs centrality of our subgraph

In [ ]:
dc_data = list(networkx.degree_centrality(subgraph_g).values())
bw_data = list(networkx.betweenness_centrality(subgraph_g).values())

import numpy as np
import matplotlib.pyplot as plt

x = dc_data
y = bw_data

plt.scatter(x, y)

#title
plt.title("Betweenness vs Centrality")

#labels
plt.xlabel("Degree Centrality")
plt.ylabel("Betweenness Centrality")
plt.show()

# Part C - Analysing Graphs

 1) Write a function that plots histgrams of the Degree of Centrality and Betweenness
     The function should take a networks graph as a parameter and return nothing
 
 2) Use the find_cliques() function to find the number of cliques in the graph 
 
 3) Based of the graph you created above, the number of cliques, and the histograms of Centrality and Betweenness, how would you discribe the community this graph represents? 
 
 Hint: Use networkx documentation to find a function that allows you to collect the attributes of a node 

In [ ]:
# Your soultion should Look somthing Like this 

def graph_data(graph):
    """
    Creates histograms of the the Degree of Centrality and the Betweenness Centrality 
    
    Parameters
    ----------
    graph - A Networkx object Graph. Graph nodes must have degree_centrality and betweenness attributes
    
    """
    # Create an empty list to hold all of our degree_centrality items  
    cen_list = []
    # Use the built in function get_node_attributes to collect the degree_centrality for each node
    for item in networkx.get_node_attributes(graph, 'degree_centrality'):
        cen_list.append(networkx.get_node_attributes(graph, 'degree_centrality')[item])
    
    # Pass the cen_list to a plt object and plot as a histogram 
    plt.figure(1)
    #title
    plt.title("Degree Centrality")

    #labels
    plt.xlabel("Degree Centrality")
    
    plt.hist(cen_list)
    
    # Create an empty list to hold all of our eigen_cent items 
    bw_list = []
    # Use the built in function get_node_attributes to collect the eigen_cent for each node
    for item in networkx.get_node_attributes(graph, 'betweenness'):
        bw_list.append(networkx.get_node_attributes(graph, 'betweenness')[item])
    
    # Pass the eig_list to a plt object and plot as a histogram 
    plt.figure(2)
    #title
    plt.title("Betweeness Centrality")

    #labels
    plt.xlabel("Betweeness Centrality")
    plt.hist(bw_list)

    
# Call the function graph_data() on our Networkx graph 
graph_data(subgraph_g)



In [ ]:
# Call the fucntion find_cliques() on our Networkx graph 
find_cliques(subgraph_g)